# RFM Analysis on Online-Retail Dataset

In [1]:
# Import libraries needed for preprocessing and visualization
import pandas as pd
import matplotlib.pyplot as plt
import ipympl
import seaborn as sns
from scipy import stats
import numpy as np
import warnings
warnings.simplefilter('ignore')


# Import libraries needed for Clustering  
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

## Read the dataset

The Online-Retail dataset from the UCI Machine Learning Repository contains information regaring online transactions made in a company from 01/12/2010 and 09/12/2011.

In [2]:
# Read the data on which analysis needs to be done and display first 5 entries
data='../input/online-retail/Online Retail.xlsx'
df_retail=pd.read_excel(data)
df_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Exploratory Data Analysis

For the purpose of RFM analysis, we only require the customer ID, number of items and its unit price, number of transactions and the date of order. 

In [3]:
# Displays datatype, number of non null values and general properties of fields
df_retail.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


We can see that number of non null values in customer ID and Description is lower than the rest of the columns, thus there exists null values in both. For puropses of customer segmentation, description is not relevant and hence the entries with null values for it need not be removed. 

In [4]:
# Number of null entries in customer ID
print("The number of null entries for CustomerID is : {}".format(df_retail['CustomerID'].isna().sum()))

The number of null entries for CustomerID is : 135080


In [5]:
# Checks the number of duplicates
print("The number of duplicate entries : {}".format(df_retail.duplicated().sum())) 

The number of duplicate entries : 5268


The country wise distribution can be helpful to further filter out data. 

In [6]:
fig, ax = plt.subplots(figsize=(6,6))
country_counts = df_retail['Country'].value_counts()
country_counts=country_counts.reset_index()
country_counts.columns=['Country','Count']
ax=sns.barplot(x='Country', y='Count',data=country_counts.head(10),estimator=max,ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
ax.set_title('Transactions in top 10 countries', fontsize = 14)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A large majority of orders are from the United Kingdom.

## Data Preprocessing
**Some insights made so far**:
- There exists 133252 null entries in CustomerID.
- There are 5268 duplicate entries.
- Most of the entries are from the UK.

First, the duplicate entries, entries with Country other than UK and entries with null values for CustomerID will be dropped.

In [7]:
# Removes all duplicates keeping the first instance
df_retail.drop_duplicates(keep='first',inplace=True) 
df_retail.duplicated().sum()

0

In [8]:
# Drop rows with null entries in CustomerID
df_retail=df_retail.dropna(subset=['CustomerID'])
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.6+ MB


In [9]:
df_retail.describe()

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


The minimum valuse of quantity and unit price are negative. This is not physically possible and could happen only due to canceled orders. According to the data description, the InvoiceNo of canceled orders contains the letter 'C' in them. These canceled order entries have to be removed to filter out actual transactions.

In [10]:
# removes all cancelled orders
print("The canceled orders are : {}".format(df_retail["InvoiceNo"].str.contains("C", na=False)))
df_retail = df_retail[~df_retail["InvoiceNo"].str.contains("C", na=False)] 

The canceled orders are : 0         False
1         False
2         False
3         False
4         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Name: InvoiceNo, Length: 401604, dtype: bool


In [11]:
df_retail.describe()

,Quantity,UnitPrice,CustomerID
count,392732.000000,392732.000000,392732.000000
mean,13.153718,3.125596,15287.734822
std,181.588420,22.240725,1713.567773
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13955.000000
50%,6.000000,1.950000,15150.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,8142.750000,18287.000000


There are still instances of negative values of unit price and quantity. These entries have to be removed.

In [12]:
# Keep only non negative values of Price and Quantity
df_retail = df_retail[(df_retail['UnitPrice']>0) & (df_retail['Quantity']>0)]
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392692 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392692 non-null  object        
 1   StockCode    392692 non-null  object        
 2   Description  392692 non-null  object        
 3   Quantity     392692 non-null  int64         
 4   InvoiceDate  392692 non-null  datetime64[ns]
 5   UnitPrice    392692 non-null  float64       
 6   CustomerID   392692 non-null  float64       
 7   Country      392692 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.0+ MB


## RFM Analysis
For the purpose of clustering, first we have to define Recency, Frequency and Monetary values.
- **Monetary value**- It can be the net amount recieved by the comany from one customer. i.e The sum of quantity times unit price across all transactions of a given customer.
- **Frequency**- This can be defined as the number of purchases made by one customer. i.e the numbver of transactions per customer.
- **Recency**- The time between last transaction and current date. Here, we can assume the most recent transaction date to be the correct date. 


In [13]:
# Calculates amount as product of unit price and Quantity
df_retail['Amount']=df_retail['Quantity']*df_retail['UnitPrice']
df_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [14]:
# Calculates total monetary value of each customers as sum of Amount of all transactions and stores in new dataframe
monetary=df_retail.groupby('CustomerID')['Amount'].sum()
monetary = monetary.reset_index()
monetary.head()

,CustomerID,Amount
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


In [15]:
# Total number of individual transaction of customers by invoice number
frequency = df_retail.groupby('CustomerID')['InvoiceNo'].count()
frequency = frequency.reset_index()
frequency.head()

,CustomerID,InvoiceNo
0,12346.0,1
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


In [16]:
# Number of days between each transaction and last recorded transaction 
df_retail['Latest'] = max(df_retail['InvoiceDate']) - df_retail['InvoiceDate']
df_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount,Latest
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,373 days 04:24:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,373 days 04:24:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00


In [17]:
# Most recent transaction per customer 
recency = df_retail.groupby('CustomerID')['Latest'].min()
recency = recency.reset_index()
recency.head()

,CustomerID,Latest
0,12346.0,325 days 02:49:00
1,12347.0,1 days 20:58:00
2,12348.0,74 days 23:37:00
3,12349.0,18 days 02:59:00
4,12350.0,309 days 20:49:00


In [18]:
# Remove time from the values
recency['Latest']=recency['Latest'].dt.days
recency.head()

,CustomerID,Latest
0,12346.0,325
1,12347.0,1
2,12348.0,74
3,12349.0,18
4,12350.0,309


In [19]:
# Merge monetary and frequency tables based on CustomerID
rfm = pd.merge(monetary, frequency, on='CustomerID', how='inner')
rfm.head()

,CustomerID,Amount,InvoiceNo
0,12346.0,77183.60,1
1,12347.0,4310.00,182
2,12348.0,1797.24,31
3,12349.0,1757.55,73
4,12350.0,334.40,17


In [20]:
# Merge monetary and frequency table with recency table based on CustomerID and change column name
rfm = pd.merge(rfm, recency, on='CustomerID', how='inner')
rfm.columns=['CustomerID','Monetary','Frequency','Recency']
rfm.head()

,CustomerID,Monetary,Frequency,Recency
0,12346.0,77183.60,1,325
1,12347.0,4310.00,182,1
2,12348.0,1797.24,31,74
3,12349.0,1757.55,73,18
4,12350.0,334.40,17,309


In [21]:
# CustomerID is stored as string
rfm['CustomerID']=rfm['CustomerID'].astype(str)

Now, we have an RFM table. For the next step, we need to remove the outliers. We will use the **z-score** method for the same.

In [22]:
# Statistical description of dataframe
rfm.describe()

,Monetary,Frequency,Recency
count,4338.000000,4338.000000,4338.000000
mean,2048.688081,90.523744,91.536422
std,8985.230220,225.506968,100.014169
min,3.750000,1.000000,0.000000
25%,306.482500,17.000000,17.000000
50%,668.570000,41.000000,50.000000
75%,1660.597500,98.000000,141.000000
max,280206.020000,7676.000000,373.000000


The mean is taken as 0 and standard deviation as 1. We will fit only the values that lie within 3 standard deviations from the mean and remove the rest. This process is repeated for all 3 columns.

In [23]:
# Set upper limit and lower limit at 3 standard deviations from the mean
m_mean=rfm['Monetary'].mean()
m_sd=rfm['Monetary'].std()
upperl_m=m_mean+3*m_sd
lowerl_m=m_mean-3*m_sd
upperl_m,lowerl_m,m_mean,m_sd

(29004.378740707227, -24907.00257934254, 2048.688080682342, 8985.230220008294)

In [24]:
# Remove outliers
rfm=rfm[(rfm['Monetary']>lowerl_m) & (rfm['Monetary']<upperl_m)] 
rfm.describe()

,Monetary,Frequency,Recency
count,4307.000000,4307.000000,4307.000000
mean,1475.632276,82.256559,91.983980
std,2460.285421,126.911127,100.037157
min,3.750000,1.000000,0.000000
25%,305.280000,17.000000,17.000000
50%,660.000000,40.000000,50.000000
75%,1622.965000,97.000000,142.000000
max,28882.440000,2677.000000,373.000000


In [25]:
# Repeat for frequency
f_mean=rfm['Frequency'].mean()
f_sd=rfm['Frequency'].std()
upperl_f=f_mean+3*f_sd
lowerl_f=f_mean-3*f_sd
upperl_f,lowerl_f,f_mean,f_sd

(462.9899401576615, -298.47682197795405, 82.25655908985372, 126.9111270226026)

In [26]:
rfm=rfm[(rfm['Frequency']>lowerl_f) & (rfm['Frequency']<upperl_f)] 
rfm.describe()

,Monetary,Frequency,Recency
count,4226.000000,4226.000000,4226.000000
mean,1354.746745,70.260530,93.543540
std,2170.353840,81.853927,100.316867
min,3.750000,1.000000,0.000000
25%,301.925000,17.000000,18.000000
50%,645.300000,39.000000,51.500000
75%,1535.212500,92.000000,146.000000
max,28754.110000,462.000000,373.000000


In [27]:
# Repeat for Recency
r_mean=rfm['Recency'].mean()
r_sd=rfm['Recency'].std()
upperl_r=r_mean+3*r_sd
lowerl_r=r_mean-3*r_sd
upperl_r,lowerl_r,r_mean,r_sd

(394.4941398007042, -207.40705981963464, 93.54353999053478, 100.31686660338981)

In [28]:
rfm.info()
rfm.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4226 entries, 1 to 4337
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  4226 non-null   object 
 1   Monetary    4226 non-null   float64
 2   Frequency   4226 non-null   int64  
 3   Recency     4226 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 165.1+ KB


,CustomerID,Monetary,Frequency,Recency
1,12347.0,4310.00,182,1
2,12348.0,1797.24,31,74
3,12349.0,1757.55,73,18
4,12350.0,334.40,17,309
5,12352.0,2506.04,85,35


## K-Means Clustering

K-Means algorithm assumes symmertical distribution and normalized variables.

- In case the variables are not symmetrically distributed, since all values are positive, we can apply log transformation.
- If the mean and variables are not equal, the variables could be normalized using Min-Max normalization. 

In [29]:
# measure skewness
def skewness_measure(df, column):
    skew = stats.skew(df[column])
    skewtest = stats.skewtest(df[column])
    plt.title('Distribution of ' + column)
    sns.distplot(df[column])
    print("{}'s: Skew: {}, : {}".format(column, skew, skewtest))
    return 

In [30]:
# Plot skewness of all variables
plt.figure(figsize=(9, 9))

plt.subplot(3, 1, 1)
skewness_measure(rfm,'Recency')

plt.subplot(3, 1, 2)
skewness_measure(rfm,'Frequency')

plt.subplot(3, 1, 3)
skewness_measure(rfm,'Monetary')

plt.tight_layout()
plt.savefig('before_transform.png', format='png', dpi=500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Recency's: Skew: 1.2184101003161467, : SkewtestResult(statistic=25.86249477653755, pvalue=1.7605318106127792e-147)
Frequency's: Skew: 2.105489814913073, : SkewtestResult(statistic=36.42611374951442, pvalue=1.6437266277039128e-290)
Monetary's: Skew: 5.023527814905374, : SkewtestResult(statistic=54.716375252126134, pvalue=0.0)


In [31]:
#log transformation
rfm_rlog = np.log(rfm['Recency']+0.1) #can't take log(0) and so add a small number
rfm_flog = np.log(rfm['Frequency'])
rfm_mlog = np.log(rfm['Monetary']+0.1)

Now let's plot the skewness after log transform.

In [32]:
log_rfm = pd.DataFrame({'Monetary': rfm_mlog,'Recency': rfm_rlog,'Frequency': rfm_flog})

# Plot skewness of all variables
plt.figure(figsize=(9, 9))

plt.subplot(3, 1, 1)
skewness_measure(log_rfm,'Recency')

plt.subplot(3, 1, 2)
skewness_measure(log_rfm,'Frequency')

plt.subplot(3, 1, 3)
skewness_measure(log_rfm,'Monetary')

plt.tight_layout()
plt.savefig('after_transform.png', format='png', dpi=500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Recency's: Skew: -1.178299413169997, : SkewtestResult(statistic=-25.26093131966803, pvalue=8.589677414023132e-141)
Frequency's: Skew: -0.4326796673140623, : SkewtestResult(statistic=-11.025655365592339, pvalue=2.87417277151095e-28)
Monetary's: Skew: 0.08954366074015034, : SkewtestResult(statistic=2.375902476486782, pvalue=0.017506088984579074)


Now, to standardize the data, we will remove mean and scale to unit variance. 

In [33]:
# Rescaling the attributes
rfm_reshaped = log_rfm[['Monetary', 'Frequency', 'Recency']]

# Instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_reshaped)
rfm_df_scaled.shape

(4226, 3)

In [34]:
# Make dataframe and name colums
rfm_scaled = pd.DataFrame(rfm_df_scaled)
rfm_scaled.columns = ['Monetary', 'Frequency', 'Recency']
rfm_scaled.head()

,Monetary,Frequency,Recency
0,1.580709,1.280348,-2.266006
1,0.836064,-0.134546,0.361425
2,0.817053,0.550089,-0.518214
3,-0.595437,-0.614784,1.252763
4,1.119086,0.671746,-0.104894


To find optimum clusters, we will plot the elbow curve and pick the k at the elbow. 

In [35]:
# Calculate SSE to find optimum number of clusters
sse = []
k_rng = range(1,11)
for k in k_rng:
    km = KMeans(n_clusters=k,init='k-means++')
    km.fit(rfm_scaled[['Monetary', 'Frequency', 'Recency']])
    sse.append(km.inertia_)

In [36]:
# Plot elbow plot to find k
fig, ax = plt.subplots()
plt.xlabel('K')
plt.ylabel('Sum of squared error')
plt.plot(k_rng,sse)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The elbow appears at 2 clusters. So, we will take k=2.

In [37]:
# Perform K-Means clustering on normalized data
km = KMeans(n_clusters=2)
y_predicted = km.fit_predict(rfm_scaled[['Monetary', 'Frequency', 'Recency']])
y_predicted

array([1, 1, 1, ..., 0, 0, 1], dtype=int32)

In [38]:
# Display cluster centers
km.cluster_centers_

array([[-0.67667525, -0.67423998,  0.51338909],
       [ 0.7966012 ,  0.79373434, -0.60437613]])

In [39]:
# Add clusters to dataset
rfm_scaled['Cluster']=y_predicted
rfm_scaled.head()

,Monetary,Frequency,Recency,Cluster
0,1.580709,1.280348,-2.266006,1
1,0.836064,-0.134546,0.361425,1
2,0.817053,0.550089,-0.518214,1
3,-0.595437,-0.614784,1.252763,0
4,1.119086,0.671746,-0.104894,1


In [40]:
rfm1=rfm_scaled[rfm_scaled['Cluster']==0]
rfm2=rfm_scaled[rfm_scaled['Cluster']==1]

Now, we can plot the clusters we obtained from K-Means clustering.

In [41]:
sns.set(style = "darkgrid")
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(rfm1.Monetary, rfm1.Frequency, rfm1.Recency, c='#58508d', label = 'Cluster 1')
ax.scatter3D(rfm2.Monetary, rfm2.Frequency, rfm2.Recency, c='#bc5090', label = 'Cluster 2')
ax.set_xlabel("Monetary",fontsize=12)
ax.set_ylabel("Frequency",fontsize=12)
ax.set_zlabel("Recency",fontsize=12)
plt.title("RFM",fontsize=18)
plt.legend(bbox_to_anchor=(1.0,1.0),prop={'size': 12})
plt.tight_layout()
plt.savefig('clusters.png', format='png', dpi=500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Conclusion

For the purpose of the following analysis, it is safe to assume that there is no correlation between the products bought and the customer cluster. The influence of products and geographical factors can be studied in a different study.

**Some reccomendations**:
- The cancelled orders can all be dealt with uniformly. In this case, while some cancelled orders were represented by the letter 'C' in the invoice number, the others were represented by negative feilds for unit price or quantity.
- Duplicates can be avoided.
- Null entries are to be reevaluated. 